In [1]:
import pandas as pd
import numpy as np
import os

Einlesen des zuvor erzeugten Datensatzes (siehe Datenaufbereitung.ipynb)

In [2]:
df = pd.read_csv('formula1_datenv1.csv', sep = ';', decimal = '.')
del df['Unnamed: 0']
df.shape

(157755, 19)

In [7]:
df.head()

,raceId,year,circuitId,grandprix_name,driverId,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructorId,constructor_name,stop_binary,total_laps,race_completion,grid,status_clean,total_milliseconds,form
0,841.0,2011.0,1.0,Australian Grand Prix,1.0,1.0,2.0,100573.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.017241,2.0,Finished,5392556.0,0.0
1,841.0,2011.0,1.0,Australian Grand Prix,1.0,2.0,2.0,93774.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.034483,2.0,Finished,5392556.0,0.0
2,841.0,2011.0,1.0,Australian Grand Prix,1.0,3.0,2.0,92900.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.051724,2.0,Finished,5392556.0,0.0
3,841.0,2011.0,1.0,Australian Grand Prix,1.0,4.0,2.0,92582.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.068966,2.0,Finished,5392556.0,0.0
4,841.0,2011.0,1.0,Australian Grand Prix,1.0,5.0,2.0,92471.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.086207,2.0,Finished,5392556.0,0.0


Anzahl der uniquen Ausprägungen in jeder Spalte

In [8]:
for col in df.columns:
    print(col, len(df[col].unique()))

raceId 137
year 7
circuitId 26
grandprix_name 25
driverId 57
lap_number 78
lap_position 24
lap_in_milliseconds 54290
driver_fullname 57
podium_position 24
constructorId 17
constructor_name 17
stop_binary 2
total_laps 18
race_completion 948
grid 25
status_clean 3
total_milliseconds 2902
form 130


In [10]:
def hot_encode_top (column, df, feat_count = 10):
    '''
    hot one encoding, limitiert auf die feat_count häufigsten features
    eines nominalen features um zu Hohe dimensionen zu vermeiden
    
    column: liste mit einem oder mehr Spaltennamen, die hot encoded werden sollen
    df: dataframe der die Datenbasis darstellt
    feat_count: Anzahl Spalten die für jede Spalte encoded werden
    '''
    df_ = df.copy(deep = True)
    
    for col in column:
        
        
        #nur die häufigsten feat_count Featues werden encoded
        encode_features = [x for x in df_[col].value_counts(ascending = False).head(feat_count).index]
        
        for feature in encode_features:
            col_feature = col + '_'+str(feature)
            #dort wo feature nicht dem encode feature entspricht wird eine 0 gesetzt
            df_[col_feature] = df_.where(df_[col] == feature, other = 0)[col]
            #encode feature selbst wird in dataframe durch eine 1 ersetzt
            df_[col_feature].replace(feature, 1, inplace = True)
        
        #löschen der nun "bereinigten" Spalte
        del df_[col]
        
    return df_

In [11]:
encoded_df = hot_encode_top(['constructorId', 'driverId', 'total_laps','year',"podium_position"], df, 5)
encoded_df.shape

(157755, 39)

In [56]:
encoded_df.head(15)

,raceId,circuitId,grandprix_name,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructor_name,stop_binary,...,total_laps_56.0,total_laps_71.0,total_laps_53.0,total_laps_70.0,total_laps_57.0,year_2012.0,year_2016.0,year_2011.0,year_2013.0,year_2014.0
0,841.0,1.0,Australian Grand Prix,1.0,2.0,100573.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,841.0,1.0,Australian Grand Prix,2.0,2.0,93774.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,841.0,1.0,Australian Grand Prix,3.0,2.0,92900.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,841.0,1.0,Australian Grand Prix,4.0,2.0,92582.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,841.0,1.0,Australian Grand Prix,5.0,2.0,92471.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,841.0,1.0,Australian Grand Prix,6.0,2.0,92434.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,841.0,1.0,Australian Grand Prix,7.0,2.0,92447.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,841.0,1.0,Australian Grand Prix,8.0,2.0,92310.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,841.0,1.0,Australian Grand Prix,9.0,2.0,92612.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,841.0,1.0,Australian Grand Prix,10.0,2.0,93121.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Hannahs zeug zum split undso

In [3]:
df = pd.read_csv('formula1_datenv1.csv', sep = ';', decimal = '.')
del df['Unnamed: 0']
df.shape

(157755, 19)

In [4]:
df.head()

,raceId,year,circuitId,grandprix_name,driverId,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructorId,constructor_name,stop_binary,total_laps,race_completion,grid,status_clean,total_milliseconds,form
0,841.0,2011.0,1.0,Australian Grand Prix,1.0,1.0,2.0,100573.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.017241,2.0,Finished,5392556.0,0.0
1,841.0,2011.0,1.0,Australian Grand Prix,1.0,2.0,2.0,93774.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.034483,2.0,Finished,5392556.0,0.0
2,841.0,2011.0,1.0,Australian Grand Prix,1.0,3.0,2.0,92900.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.051724,2.0,Finished,5392556.0,0.0
3,841.0,2011.0,1.0,Australian Grand Prix,1.0,4.0,2.0,92582.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.068966,2.0,Finished,5392556.0,0.0
4,841.0,2011.0,1.0,Australian Grand Prix,1.0,5.0,2.0,92471.0,Lewis Hamilton,2.0,1.0,McLaren,0.0,58.0,0.086207,2.0,Finished,5392556.0,0.0


# NICHT AUSFÜHREN BRAUCHT EWIG

In [5]:
drivers = df.driverId.tolist()
#Kategorische spalten in dummievariablen umwandeln
df = pd.get_dummies(df, columns=['circuitId', 'driverId', 'constructorId', 'status_clean'])
df['driverId']= drivers
#dictionary enthält einzelne datensätze zu jedem rennen
split_by_race = {}
#grenzwert ab dem vorhersage über rennausgang gemacht werden soll (orientiert sich an race_completion)
border = 0.5
#dictionary, welches renndatensätze nur bis zu einem gewissen zeitpunkt enthält (zp wird nach border gewählt)
sliced_races = {}

for rid in df['raceId'].unique():
    race = df.where(df.raceId == rid).dropna(how = 'all')
    race.reset_index(inplace = True, drop = True)
    split_by_race[rid] = race
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    
    #hinzufügen einer spalte die die insgesamt gefahrenen millisekunden enthält
    #bis zu dieser Runde pro Fahrer enthält
    race_shortened['sum_milliseconds_pro_lap'] = 0
    
    for did in race_shortened.driverId.unique():
        for lapnum in race_shortened.lap_number.unique():
            sum_ms = np.sum(race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number<=lapnum)).dropna(how = 'all')['lap_in_milliseconds'])
            
            #setzen der bisher gefahrenen Zeit (kumuliert) pro Fahrer und Runde
            race_shortened.loc[race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number==lapnum)).dropna(how = 'all').index,'sum_milliseconds_pro_lap'] = sum_ms
    
    sliced_races[rid] = race_shortened
    
    
    

In [6]:
if not os.path.exists('sliced_data'):
    os.makedirs('sliced_data')
    
if not os.path.exists('split_data'):
    os.makedirs('split_data')

for key, value in sliced_races.items():
    name = 'sliced_data/sliced_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')
for key, value in split_by_race.items():
    name = 'split_data/split_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')

In [58]:
sliced_races[841].shape

(789, 120)

In [2]:
rain_id = [847,861,879,910,914,934,942,953,957,967,970,982]
if os.path.exists('sliced_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/sliced_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    sliced_races = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('sliced_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('sliced_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        df["rain"] = 0
        if list(df["raceId"])[0] in rain_id:
            df["rain"] = 1
        sliced_races[f] = df
    print('Einlesen der sliced Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
    
if os.path.exists('split_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/split_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    split_by_race = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        split_by_race[f] = df
    print('Einlesen der split Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')

Einlesen der sliced Dateien erfolgreich
Einlesen der split Dateien erfolgreich


In [25]:
(sliced_races.keys())

dict_keys([841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 926, 927, 928, 929, 930, 931, 932, 933, 934, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988])

In [38]:
sliced_races[988]

,raceId,year,grandprix_name,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructor_name,stop_binary,...,constructorId_207.0,constructorId_208.0,constructorId_209.0,constructorId_210.0,status_clean_DNF,status_clean_Finished,status_clean_lapped,driverId,sum_milliseconds_pro_lap,rain
0,988.0,2017.0,Abu Dhabi Grand Prix,1.0,11.0,113969.0,Fernando Alonso,9.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,113969.0,0
1,988.0,2017.0,Abu Dhabi Grand Prix,2.0,11.0,107148.0,Fernando Alonso,9.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,221117.0,0
2,988.0,2017.0,Abu Dhabi Grand Prix,3.0,11.0,105895.0,Fernando Alonso,9.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,327012.0,0
3,988.0,2017.0,Abu Dhabi Grand Prix,4.0,11.0,105576.0,Fernando Alonso,9.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,432588.0,0
4,988.0,2017.0,Abu Dhabi Grand Prix,5.0,11.0,105504.0,Fernando Alonso,9.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,538092.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,988.0,2017.0,Abu Dhabi Grand Prix,23.0,19.0,124783.0,Kevin Magnussen,13.0,Haas F1 Team,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,825.0,2478635.0,0
529,988.0,2017.0,Abu Dhabi Grand Prix,24.0,19.0,104938.0,Kevin Magnussen,13.0,Haas F1 Team,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,825.0,2583573.0,0
530,988.0,2017.0,Abu Dhabi Grand Prix,25.0,19.0,104785.0,Kevin Magnussen,13.0,Haas F1 Team,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,825.0,2688358.0,0
531,988.0,2017.0,Abu Dhabi Grand Prix,26.0,19.0,104990.0,Kevin Magnussen,13.0,Haas F1 Team,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,825.0,2793348.0,0


In [114]:
temp = split_by_race[911]

In [115]:
temp

,raceId,year,grandprix_name,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructor_name,stop_binary,...,constructorId_205.0,constructorId_206.0,constructorId_207.0,constructorId_208.0,constructorId_209.0,constructorId_210.0,status_clean_DNF,status_clean_Finished,status_clean_lapped,driverId
0,911.0,2014.0,Belgian Grand Prix,1.0,10.0,123675.0,Jenson Button,6.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
1,911.0,2014.0,Belgian Grand Prix,2.0,9.0,118275.0,Jenson Button,6.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
2,911.0,2014.0,Belgian Grand Prix,3.0,8.0,116999.0,Jenson Button,6.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
3,911.0,2014.0,Belgian Grand Prix,4.0,8.0,116803.0,Jenson Button,6.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
4,911.0,2014.0,Belgian Grand Prix,5.0,8.0,116559.0,Jenson Button,6.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,911.0,2014.0,Belgian Grand Prix,35.0,19.0,119272.0,Jules Bianchi,18.0,Marussia,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,824.0
854,911.0,2014.0,Belgian Grand Prix,36.0,19.0,117850.0,Jules Bianchi,18.0,Marussia,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,824.0
855,911.0,2014.0,Belgian Grand Prix,37.0,19.0,116708.0,Jules Bianchi,18.0,Marussia,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,824.0
856,911.0,2014.0,Belgian Grand Prix,38.0,19.0,117388.0,Jules Bianchi,18.0,Marussia,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,824.0


In [29]:
#setzen der prozentualen Grenze, ab der weiterer Verlauf des Rennens vorhergesagt werden soll
border = 0.7

for race in split_by_race.values():
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    

In [24]:
temp.where(temp.race_completion == min(temp.where(temp.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]

41.0

# ANN Try

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random

In [12]:
x = temp[['year', 'circuitId', 'driverId',
       'lap_number', 'lap_position', 'lap_in_milliseconds', 'driver_fullname',
       'constructorId', 'stop_binary',
       'race_completion', 'grid', 'status_clean',
       'form']]
y = temp['podium_position']

In [25]:
encoded_df = hot_encode_top(['circuitId','constructorId', 'driverId', 'year', "status_clean"], x, 5)
encoded_df.shape

(1083, 26)

In [26]:
print(encoded_df.columns)
encoded_df.head()

Index(['raceId', 'grandprix_name', 'lap_number', 'lap_position',
       'lap_in_milliseconds', 'driver_fullname', 'constructor_name',
       'stop_binary', 'race_completion', 'grid', 'form', 'circuitId_1.0',
       'constructorId_15.0', 'constructorId_9.0', 'constructorId_6.0',
       'constructorId_1.0', 'constructorId_4.0', 'driverId_1.0',
       'driverId_155.0', 'driverId_18.0', 'driverId_808.0', 'driverId_815.0',
       'year_2011.0', 'status_clean_Finished', 'status_clean_lapped',
       'status_clean_DNF'],
      dtype='object')


,raceId,grandprix_name,lap_number,lap_position,lap_in_milliseconds,driver_fullname,constructor_name,stop_binary,race_completion,grid,...,constructorId_4.0,driverId_1.0,driverId_155.0,driverId_18.0,driverId_808.0,driverId_815.0,year_2011.0,status_clean_Finished,status_clean_lapped,status_clean_DNF
0,841.0,Australian Grand Prix,1.0,2.0,100573.0,Lewis Hamilton,McLaren,0.0,0.017241,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0,0
1,841.0,Australian Grand Prix,2.0,2.0,93774.0,Lewis Hamilton,McLaren,0.0,0.034483,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0,0
2,841.0,Australian Grand Prix,3.0,2.0,92900.0,Lewis Hamilton,McLaren,0.0,0.051724,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0,0
3,841.0,Australian Grand Prix,4.0,2.0,92582.0,Lewis Hamilton,McLaren,0.0,0.068966,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0,0
4,841.0,Australian Grand Prix,5.0,2.0,92471.0,Lewis Hamilton,McLaren,0.0,0.086207,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0,0


In [27]:
x_tensor = torch.tensor(encoded_df[['lap_number', 'lap_position',
       'lap_in_milliseconds',
       'stop_binary', 'race_completion', 'grid', 'form', 'circuitId_1.0',
       'constructorId_15.0', 'constructorId_9.0', 'constructorId_6.0',
       'constructorId_1.0', 'constructorId_4.0', 'driverId_1.0',
       'driverId_155.0', 'driverId_18.0', 'driverId_808.0', 'driverId_815.0',
       'year_2011.0', 'status_clean_Finished', 'status_clean_lapped',
       'status_clean_DNF']].values)

In [32]:
x_tensor.size()

torch.Size([1083, 22])

In [43]:
lin1 = nn.Linear(22, 40)
lin1(x_tensor.float())

tensor([[ 15269.8994,  -3026.0515,  -2752.2817,  ...,  -9950.7588,
         -10450.6045, -12089.6338],
        [ 14237.4775,  -2821.4431,  -2566.3311,  ...,  -9278.1924,
          -9744.3271, -11272.1572],
        [ 14104.6475,  -2795.1162,  -2542.5476,  ...,  -9191.8955,
          -9653.7158, -11166.8906],
        ...,
        [ 14452.6826,  -2863.9412,  -2611.0034,  ...,  -9426.6699,
          -9907.6748, -11442.5918],
        [ 14586.6182,  -2890.4822,  -2635.3091,  ...,  -9514.2246,
          -9999.6348, -11548.5244],
        [ 14954.8271,  -2963.4519,  -2701.8469,  ...,  -9754.4561,
         -10251.9287, -11839.9326]], grad_fn=<AddmmBackward>)

[841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 926, 927, 928, 929, 930, 931, 932, 933, 934, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988]

## Training and Test Data split and preparation

In [109]:
train_data = []
test_data = []
test_final = {}
temp_y_podium = []
test_races = list(sliced_races.keys())
random.shuffle(test_races)
test_races = test_races[0:5]
nogo_columns_hannah_kacke = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary','constructorId_1.0',
                 'constructorId_3.0',
                 'constructorId_4.0',
                 'constructorId_5.0',
                 'constructorId_6.0',
                 'constructorId_9.0',
                 'constructorId_10.0',
                 'constructorId_15.0',
                 'constructorId_131.0',
                 'constructorId_164.0',
                 'constructorId_166.0',
                 'constructorId_205.0',
                 'constructorId_206.0',
                 'constructorId_207.0',
                 'constructorId_208.0',
                 'constructorId_209.0',
                 'constructorId_210.0']
nogo_columns = ['year', 'podium_position', 'raceId','lap_number','total_laps','driverId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds']
for key, value in sliced_races.items():
    helper = key
    for did in value.driverId.unique():
        temp = value.where(value.driverId == did).dropna(how = "all")
        if list(temp["podium_position"])[0] < 0: #Top x finish positions
            pp = 1
        else:
            if key in test_races:
                temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                #temp_y = temp_y[0]
                cols = [col for col in temp.columns if col not in nogo_columns]
                temp_x = temp[cols]
                stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                temp_x = temp_x.tail(1)
                temp_x['stop_binary'] = stops
                x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                #temp_x = x_tensor.float()
                test_data.append((x_tensor, [temp_y[0]]))
            else:
                temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                #temp_y = temp_y[0]
                cols = [col for col in temp.columns if col not in nogo_columns]
                temp_x = temp[cols]
                stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                temp_x = temp_x.tail(1)
                temp_x['stop_binary'] = stops
                x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                #temp_x = x_tensor.float()
                train_data.append((x_tensor, [temp_y[0]]))
    if key in test_races:
        test_final[key]=test_data
    test_data = []
random.shuffle(train_data)
#random.shuffle(test_data)
#test_data = train_data[len(train_data)-100:]
train_data = train_data#[0:len(train_data)-100]
        
        #break
        #for i, row in temp.iterrows():

## Batch ansatz

In [104]:
train_data = []
train_data_x = []
train_data_y = []
test_data = []
count = 0
nogo_columns_hannah = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary','constructorId_1.0',
                 'constructorId_3.0',
                 'constructorId_4.0',
                 'constructorId_5.0',
                 'constructorId_6.0',
                 'constructorId_9.0',
                 'constructorId_10.0',
                 'constructorId_15.0',
                 'constructorId_131.0',
                 'constructorId_164.0',
                 'constructorId_166.0',
                 'constructorId_205.0',
                 'constructorId_206.0',
                 'constructorId_207.0',
                 'constructorId_208.0',
                 'constructorId_209.0',
                 'constructorId_210.0']
nogo_columns = ['year', 'podium_position', 'raceId','lap_number','total_laps','driverId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds']
for key, value in sliced_races.items():
    for did in value.driverId.unique():
        temp = value.where(value.driverId == did).dropna(how = "all")
        if list(temp["podium_position"])[0] < 0: #Top x finish positions
            pp = 1
        else:
            if key == 988:
                temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                #temp_y = temp_y[0]
                cols = [col for col in temp.columns if col not in nogo_columns]
                temp_x = temp[cols]
                stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                temp_x = temp_x.tail(1)
                temp_x['stop_binary'] = stops
                x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                test_data.append((x_tensor, [temp_y[0]]))
                
            else:
                temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                #temp_y = temp_y[0]
                cols = [col for col in temp.columns if col not in nogo_columns]
                temp_x = temp[cols]
                stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                temp_x = temp_x.tail(1)
                temp_x['stop_binary'] = stops
                x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                #temp_x = x_tensor.float()
                #train_data.append((x_tensor, [temp_y[0]]))
                train_data_x.append(x_tensor)
                train_data_y.append([temp_y[0]])
            
    #print(count)
    #count +=1
    if count == 137:
        break
    train_data.append((torch.stack(train_data_x), train_data_y))
    
    train_data_x = []
    train_data_y = []
print("hier", len(train_data))            
random.shuffle(train_data)
random.shuffle(test_data)
#test_data = train_data[len(train_data)-100:]
train_data = train_data#[0:len(train_data)-100]
print("hier", len(train_data))            
        #break
        #for i, row in temp.iterrows():

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
hier 136
hier 136


In [87]:
train_data[0]

(tensor([[[ 2.0000,  7.0000,  0.4828,  ...,  0.0000, 43.6360,  0.0000]],
 
         [[ 7.0000,  7.0000,  0.4828,  ...,  0.0000, 44.5796,  0.0000]],
 
         [[15.0000,  7.0000,  0.4828,  ...,  0.0000, 45.8120,  0.0000]],
 
         ...,
 
         [[16.0000,  7.0000,  0.4828,  ...,  1.0000, 45.9951,  0.0000]],
 
         [[18.0000,  7.0000,  0.4828,  ...,  0.0000, 56.5002,  0.0000]],
 
         [[17.0000,  7.0000,  0.4828,  ...,  0.0000, 46.9338,  0.0000]]],
        dtype=torch.float64),
 [[2.0],
  [6.0],
  [24.0],
  [24.0],
  [12.0],
  [3.0],
  [8.0],
  [11.0],
  [4.0],
  [7.0],
  [9.0],
  [10.0],
  [5.0],
  [1.0],
  [24.0],
  [24.0],
  [24.0],
  [24.0],
  [24.0],
  [13.0],
  [24.0],
  [14.0]])

In [217]:
temp_x.sum(axis = 0)[2]

0.0

In [40]:
temp[cols]

,lap_position,stop_binary,race_completion,grid,form,circuitId_1.0,circuitId_2.0,circuitId_3.0,circuitId_4.0,circuitId_5.0,...,constructorId_206.0,constructorId_207.0,constructorId_208.0,constructorId_209.0,constructorId_210.0,status_clean_DNF,status_clean_Finished,status_clean_lapped,sum_milliseconds_pro_lap,rain
506,20.0,0.0,0.018182,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,120490.0,0.0
507,20.0,0.0,0.036364,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,229189.0,0.0
508,20.0,0.0,0.054545,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,336592.0,0.0
509,20.0,0.0,0.072727,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,443352.0,0.0
510,20.0,0.0,0.090909,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,550549.0,0.0
511,20.0,0.0,0.109091,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,657506.0,0.0
512,19.0,0.0,0.127273,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,763850.0,0.0
513,19.0,0.0,0.145455,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,870174.0,0.0
514,19.0,0.0,0.163636,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,976417.0,0.0
515,19.0,0.0,0.181818,14.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1082838.0,0.0


In [42]:
train_data[0][0][0][0]

tensor(5., dtype=torch.float64)

# YAYYYYY

In [ ]:
train_data = []
test_data = []
test_final = {}
temp_y_podium = []
test_races = list(sliced_races.keys())
random.shuffle(test_races)
test_races = test_races[0:5]
nogo_columns_hannah_kacke = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary','constructorId_1.0',
                 'constructorId_3.0',
                 'constructorId_4.0',
                 'constructorId_5.0',
                 'constructorId_6.0',
                 'constructorId_9.0',
                 'constructorId_10.0',
                 'constructorId_15.0',
                 'constructorId_131.0',
                 'constructorId_164.0',
                 'constructorId_166.0',
                 'constructorId_205.0',
                 'constructorId_206.0',
                 'constructorId_207.0',
                 'constructorId_208.0',
                 'constructorId_209.0',
                 'constructorId_210.0']
nogo_columns = ['year', 'podium_position', 'raceId','lap_number','total_laps','driverId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds']
for key, value in sliced_races.items():
    helper = key
    for did in value.driverId.unique():
        temp = value.where(value.driverId == did).dropna(how = "all")
        if list(temp["podium_position"])[0] < 0: #Top x finish positions
            pp = 1
        else:
            if key in test_races:
                temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                #temp_y = temp_y[0]
                cols = [col for col in temp.columns if col not in nogo_columns]
                temp_x = temp[cols]
                stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                temp_x = temp_x.tail(1)
                temp_x['stop_binary'] = stops
                x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                #temp_x = x_tensor.float()
                test_data.append((x_tensor, [temp_y[0]]))
            else:
                temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                #temp_y = temp_y[0]
                cols = [col for col in temp.columns if col not in nogo_columns]
                temp_x = temp[cols]
                stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                temp_x = temp_x.tail(1)
                temp_x['stop_binary'] = stops
                x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                #temp_x = x_tensor.float()
                train_data.append((x_tensor, [temp_y[0]]))
    if key in test_races:
        test_final[key]=test_data
    test_data = []
random.shuffle(train_data)
#random.shuffle(test_data)
#test_data = train_data[len(train_data)-100:]
train_data = train_data#[0:len(train_data)-100]
        
        #break
        #for i, row in temp.iterrows():

In [79]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        self.fc1 = nn.Linear(110, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 190)
        self.fc4 = nn.Linear(190, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        x = self.dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        #x = self.dropout(x)
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x
        

In [80]:

model = Netz()
model.cuda()
        

optimizer = optim.Adam(model.parameters(), lr = 0.0001)        
def train(epoch):
    model.train()
    batch_id = 0
    for data, target in train_data:
        data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0).cuda()
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        optimizer.zero_grad()
        out = model(data)
        #print("Out: ", out, out.size())
        #print("Target: ", target, target.size())
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_id *len(data), len(train_data),
        100. * batch_id / len(train_data), loss.item()))
        batch_id +=1
        
    #random.shuffle(train_data)
        

In [81]:
for epoch in range(1,10):
    train(epoch)


Train Epoche: 1 [0/2793 (0%)]	Loss: 492.434814
Train Epoche: 1 [1/2793 (0%)]	Loss: 26.953854
Train Epoche: 1 [2/2793 (0%)]	Loss: 230.778763
Train Epoche: 1 [3/2793 (0%)]	Loss: 173.960419
Train Epoche: 1 [4/2793 (0%)]	Loss: 125.160843
Train Epoche: 1 [5/2793 (0%)]	Loss: 295.658234
Train Epoche: 1 [6/2793 (0%)]	Loss: 51.642220
Train Epoche: 1 [7/2793 (0%)]	Loss: 66.947922
Train Epoche: 1 [8/2793 (0%)]	Loss: 173.812927
Train Epoche: 1 [9/2793 (0%)]	Loss: 26.865902
Train Epoche: 1 [10/2793 (0%)]	Loss: 491.987183
Train Epoche: 1 [11/2793 (0%)]	Loss: 17.494171
Train Epoche: 1 [12/2793 (0%)]	Loss: 38.214256
Train Epoche: 1 [13/2793 (0%)]	Loss: 124.936607
Train Epoche: 1 [14/2793 (1%)]	Loss: 261.791351
Train Epoche: 1 [15/2793 (1%)]	Loss: 124.963432
Train Epoche: 1 [16/2793 (1%)]	Loss: 125.030434
Train Epoche: 1 [17/2793 (1%)]	Loss: 103.483902
Train Epoche: 1 [18/2793 (1%)]	Loss: 200.876236
Train Epoche: 1 [19/2793 (1%)]	Loss: 17.406805
Train Epoche: 1 [20/2793 (1%)]	Loss: 491.542633
Train Epo

Train Epoche: 1 [171/2793 (6%)]	Loss: 43.687405
Train Epoche: 1 [172/2793 (6%)]	Loss: 72.289482
Train Epoche: 1 [173/2793 (6%)]	Loss: 4.242140
Train Epoche: 1 [174/2793 (6%)]	Loss: 105.988968
Train Epoche: 1 [175/2793 (6%)]	Loss: 15.321520
Train Epoche: 1 [176/2793 (6%)]	Loss: 6.421694
Train Epoche: 1 [177/2793 (6%)]	Loss: 14.602064
Train Epoche: 1 [178/2793 (6%)]	Loss: 0.440638
Train Epoche: 1 [179/2793 (6%)]	Loss: 82.124275
Train Epoche: 1 [180/2793 (6%)]	Loss: 15.789227
Train Epoche: 1 [181/2793 (6%)]	Loss: 67.204292
Train Epoche: 1 [182/2793 (7%)]	Loss: 17.611988
Train Epoche: 1 [183/2793 (7%)]	Loss: 0.302664
Train Epoche: 1 [184/2793 (7%)]	Loss: 45.487988
Train Epoche: 1 [185/2793 (7%)]	Loss: 0.000129
Train Epoche: 1 [186/2793 (7%)]	Loss: 52.863277
Train Epoche: 1 [187/2793 (7%)]	Loss: 4.887486
Train Epoche: 1 [188/2793 (7%)]	Loss: 16.459127
Train Epoche: 1 [189/2793 (7%)]	Loss: 0.604231
Train Epoche: 1 [190/2793 (7%)]	Loss: 7.092443
Train Epoche: 1 [191/2793 (7%)]	Loss: 56.254677

Train Epoche: 1 [340/2793 (12%)]	Loss: 43.762245
Train Epoche: 1 [341/2793 (12%)]	Loss: 58.053802
Train Epoche: 1 [342/2793 (12%)]	Loss: 100.835381
Train Epoche: 1 [343/2793 (12%)]	Loss: 53.507027
Train Epoche: 1 [344/2793 (12%)]	Loss: 209.303665
Train Epoche: 1 [345/2793 (12%)]	Loss: 43.443161
Train Epoche: 1 [346/2793 (12%)]	Loss: 68.388222
Train Epoche: 1 [347/2793 (12%)]	Loss: 29.952242
Train Epoche: 1 [348/2793 (12%)]	Loss: 49.913921
Train Epoche: 1 [349/2793 (12%)]	Loss: 65.717812
Train Epoche: 1 [350/2793 (13%)]	Loss: 15.415952
Train Epoche: 1 [351/2793 (13%)]	Loss: 65.032700
Train Epoche: 1 [352/2793 (13%)]	Loss: 0.139772
Train Epoche: 1 [353/2793 (13%)]	Loss: 0.000012
Train Epoche: 1 [354/2793 (13%)]	Loss: 2.757101
Train Epoche: 1 [355/2793 (13%)]	Loss: 27.572033
Train Epoche: 1 [356/2793 (13%)]	Loss: 44.787643
Train Epoche: 1 [357/2793 (13%)]	Loss: 1.302690
Train Epoche: 1 [358/2793 (13%)]	Loss: 68.115273
Train Epoche: 1 [359/2793 (13%)]	Loss: 12.276804
Train Epoche: 1 [360/2

Train Epoche: 1 [507/2793 (18%)]	Loss: 121.545845
Train Epoche: 1 [508/2793 (18%)]	Loss: 11.645894
Train Epoche: 1 [509/2793 (18%)]	Loss: 2.463100
Train Epoche: 1 [510/2793 (18%)]	Loss: 19.214062
Train Epoche: 1 [511/2793 (18%)]	Loss: 28.809793
Train Epoche: 1 [512/2793 (18%)]	Loss: 4.797013
Train Epoche: 1 [513/2793 (18%)]	Loss: 5.311233
Train Epoche: 1 [514/2793 (18%)]	Loss: 0.372954
Train Epoche: 1 [515/2793 (18%)]	Loss: 2.366025
Train Epoche: 1 [516/2793 (18%)]	Loss: 51.839542
Train Epoche: 1 [517/2793 (19%)]	Loss: 29.736729
Train Epoche: 1 [518/2793 (19%)]	Loss: 0.819095
Train Epoche: 1 [519/2793 (19%)]	Loss: 14.510511
Train Epoche: 1 [520/2793 (19%)]	Loss: 7.347776
Train Epoche: 1 [521/2793 (19%)]	Loss: 10.548365
Train Epoche: 1 [522/2793 (19%)]	Loss: 53.360844
Train Epoche: 1 [523/2793 (19%)]	Loss: 0.194452
Train Epoche: 1 [524/2793 (19%)]	Loss: 0.763454
Train Epoche: 1 [525/2793 (19%)]	Loss: 1.048178
Train Epoche: 1 [526/2793 (19%)]	Loss: 0.351360
Train Epoche: 1 [527/2793 (19%

Train Epoche: 1 [675/2793 (24%)]	Loss: 26.084473
Train Epoche: 1 [676/2793 (24%)]	Loss: 37.070030
Train Epoche: 1 [677/2793 (24%)]	Loss: 0.016234
Train Epoche: 1 [678/2793 (24%)]	Loss: 36.098522
Train Epoche: 1 [679/2793 (24%)]	Loss: 202.580948
Train Epoche: 1 [680/2793 (24%)]	Loss: 22.404619
Train Epoche: 1 [681/2793 (24%)]	Loss: 18.255960
Train Epoche: 1 [682/2793 (24%)]	Loss: 19.607670
Train Epoche: 1 [683/2793 (24%)]	Loss: 0.089490
Train Epoche: 1 [684/2793 (24%)]	Loss: 9.033344
Train Epoche: 1 [685/2793 (25%)]	Loss: 66.983192
Train Epoche: 1 [686/2793 (25%)]	Loss: 0.022564
Train Epoche: 1 [687/2793 (25%)]	Loss: 1.691317
Train Epoche: 1 [688/2793 (25%)]	Loss: 22.119249
Train Epoche: 1 [689/2793 (25%)]	Loss: 37.836784
Train Epoche: 1 [690/2793 (25%)]	Loss: 28.051197
Train Epoche: 1 [691/2793 (25%)]	Loss: 11.323339
Train Epoche: 1 [692/2793 (25%)]	Loss: 195.161224
Train Epoche: 1 [693/2793 (25%)]	Loss: 29.000097
Train Epoche: 1 [694/2793 (25%)]	Loss: 9.895743
Train Epoche: 1 [695/279

...

Train Epoche: 9 [2098/2793 (75%)]	Loss: 20.579962
Train Epoche: 9 [2099/2793 (75%)]	Loss: 0.605481
Train Epoche: 9 [2100/2793 (75%)]	Loss: 19.983179
Train Epoche: 9 [2101/2793 (75%)]	Loss: 0.125549
Train Epoche: 9 [2102/2793 (75%)]	Loss: 1.091049
Train Epoche: 9 [2103/2793 (75%)]	Loss: 0.872932
Train Epoche: 9 [2104/2793 (75%)]	Loss: 66.534386
Train Epoche: 9 [2105/2793 (75%)]	Loss: 7.611948
Train Epoche: 9 [2106/2793 (75%)]	Loss: 2.015520
Train Epoche: 9 [2107/2793 (75%)]	Loss: 5.391474
Train Epoche: 9 [2108/2793 (75%)]	Loss: 0.425844
Train Epoche: 9 [2109/2793 (76%)]	Loss: 1.262815
Train Epoche: 9 [2110/2793 (76%)]	Loss: 2.274042
Train Epoche: 9 [2111/2793 (76%)]	Loss: 0.095575
Train Epoche: 9 [2112/2793 (76%)]	Loss: 62.619827
Train Epoche: 9 [2113/2793 (76%)]	Loss: 0.055591
Train Epoche: 9 [2114/2793 (76%)]	Loss: 9.212428
Train Epoche: 9 [2115/2793 (76%)]	Loss: 2.494502
Train Epoche: 9 [2116/2793 (76%)]	Loss: 0.955005
Train Epoche: 9 [2117/2793 (76%)]	Loss: 3.214679
Train Epoche: 9 

Train Epoche: 9 [2264/2793 (81%)]	Loss: 0.068164
Train Epoche: 9 [2265/2793 (81%)]	Loss: 4.368507
Train Epoche: 9 [2266/2793 (81%)]	Loss: 0.071649
Train Epoche: 9 [2267/2793 (81%)]	Loss: 1.135888
Train Epoche: 9 [2268/2793 (81%)]	Loss: 0.157743
Train Epoche: 9 [2269/2793 (81%)]	Loss: 6.865310
Train Epoche: 9 [2270/2793 (81%)]	Loss: 0.218415
Train Epoche: 9 [2271/2793 (81%)]	Loss: 12.462410
Train Epoche: 9 [2272/2793 (81%)]	Loss: 2.568556
Train Epoche: 9 [2273/2793 (81%)]	Loss: 4.998956
Train Epoche: 9 [2274/2793 (81%)]	Loss: 1.440503
Train Epoche: 9 [2275/2793 (81%)]	Loss: 5.794244
Train Epoche: 9 [2276/2793 (81%)]	Loss: 7.751593
Train Epoche: 9 [2277/2793 (82%)]	Loss: 0.573780
Train Epoche: 9 [2278/2793 (82%)]	Loss: 0.015098
Train Epoche: 9 [2279/2793 (82%)]	Loss: 1.959481
Train Epoche: 9 [2280/2793 (82%)]	Loss: 1.027321
Train Epoche: 9 [2281/2793 (82%)]	Loss: 51.419624
Train Epoche: 9 [2282/2793 (82%)]	Loss: 0.055583
Train Epoche: 9 [2283/2793 (82%)]	Loss: 10.312160
Train Epoche: 9 [

Train Epoche: 9 [2430/2793 (87%)]	Loss: 10.122861
Train Epoche: 9 [2431/2793 (87%)]	Loss: 2.307393
Train Epoche: 9 [2432/2793 (87%)]	Loss: 0.621876
Train Epoche: 9 [2433/2793 (87%)]	Loss: 14.455690
Train Epoche: 9 [2434/2793 (87%)]	Loss: 1.678028
Train Epoche: 9 [2435/2793 (87%)]	Loss: 4.016478
Train Epoche: 9 [2436/2793 (87%)]	Loss: 6.076282
Train Epoche: 9 [2437/2793 (87%)]	Loss: 1.891018
Train Epoche: 9 [2438/2793 (87%)]	Loss: 1.187149
Train Epoche: 9 [2439/2793 (87%)]	Loss: 4.034332
Train Epoche: 9 [2440/2793 (87%)]	Loss: 5.645002
Train Epoche: 9 [2441/2793 (87%)]	Loss: 0.105856
Train Epoche: 9 [2442/2793 (87%)]	Loss: 5.679775
Train Epoche: 9 [2443/2793 (87%)]	Loss: 15.174179
Train Epoche: 9 [2444/2793 (88%)]	Loss: 0.003261
Train Epoche: 9 [2445/2793 (88%)]	Loss: 0.219381
Train Epoche: 9 [2446/2793 (88%)]	Loss: 31.721411
Train Epoche: 9 [2447/2793 (88%)]	Loss: 0.012283
Train Epoche: 9 [2448/2793 (88%)]	Loss: 16.047092
Train Epoche: 9 [2449/2793 (88%)]	Loss: 4.475802
Train Epoche: 9

Train Epoche: 9 [2596/2793 (93%)]	Loss: 3.613631
Train Epoche: 9 [2597/2793 (93%)]	Loss: 6.416876
Train Epoche: 9 [2598/2793 (93%)]	Loss: 14.585579
Train Epoche: 9 [2599/2793 (93%)]	Loss: 17.231031
Train Epoche: 9 [2600/2793 (93%)]	Loss: 0.267237
Train Epoche: 9 [2601/2793 (93%)]	Loss: 2.638766
Train Epoche: 9 [2602/2793 (93%)]	Loss: 0.046558
Train Epoche: 9 [2603/2793 (93%)]	Loss: 1.499326
Train Epoche: 9 [2604/2793 (93%)]	Loss: 18.988930
Train Epoche: 9 [2605/2793 (93%)]	Loss: 0.005503
Train Epoche: 9 [2606/2793 (93%)]	Loss: 5.619788
Train Epoche: 9 [2607/2793 (93%)]	Loss: 0.947814
Train Epoche: 9 [2608/2793 (93%)]	Loss: 11.569182
Train Epoche: 9 [2609/2793 (93%)]	Loss: 0.223729
Train Epoche: 9 [2610/2793 (93%)]	Loss: 13.286877
Train Epoche: 9 [2611/2793 (93%)]	Loss: 2.081918
Train Epoche: 9 [2612/2793 (94%)]	Loss: 5.089624
Train Epoche: 9 [2613/2793 (94%)]	Loss: 6.991382
Train Epoche: 9 [2614/2793 (94%)]	Loss: 6.319582
Train Epoche: 9 [2615/2793 (94%)]	Loss: 4.135431
Train Epoche: 9

Train Epoche: 9 [2762/2793 (99%)]	Loss: 0.240257
Train Epoche: 9 [2763/2793 (99%)]	Loss: 9.921788
Train Epoche: 9 [2764/2793 (99%)]	Loss: 0.130663
Train Epoche: 9 [2765/2793 (99%)]	Loss: 34.095287
Train Epoche: 9 [2766/2793 (99%)]	Loss: 1.445389
Train Epoche: 9 [2767/2793 (99%)]	Loss: 1.564124
Train Epoche: 9 [2768/2793 (99%)]	Loss: 9.633005
Train Epoche: 9 [2769/2793 (99%)]	Loss: 0.382095
Train Epoche: 9 [2770/2793 (99%)]	Loss: 7.606729
Train Epoche: 9 [2771/2793 (99%)]	Loss: 0.090257
Train Epoche: 9 [2772/2793 (99%)]	Loss: 0.863482
Train Epoche: 9 [2773/2793 (99%)]	Loss: 0.430179
Train Epoche: 9 [2774/2793 (99%)]	Loss: 40.798630
Train Epoche: 9 [2775/2793 (99%)]	Loss: 14.092127
Train Epoche: 9 [2776/2793 (99%)]	Loss: 16.732424
Train Epoche: 9 [2777/2793 (99%)]	Loss: 0.003382
Train Epoche: 9 [2778/2793 (99%)]	Loss: 3.020680
Train Epoche: 9 [2779/2793 (99%)]	Loss: 0.586539
Train Epoche: 9 [2780/2793 (100%)]	Loss: 0.009655
Train Epoche: 9 [2781/2793 (100%)]	Loss: 3.109119
Train Epoche: 

### Total Loss werte
- Die Fahrer unter den ersten 4 halten meistens ihre Position nach 50% des rennens -> wäre besser das vorher zu sagen

In [111]:
total_results[0][904] == total_results[0][911]

False

In [110]:
total_results = test_times(test_final)
total_results

845
tensor([[2.0313]], grad_fn=<CopyBackwards>)
Output:  [[2.0312984]]
Target:  [[2.]]
Difference:  [[0.0312984]]
tensor([[3.4460]], grad_fn=<CopyBackwards>)
Output:  [[3.4460022]]
Target:  [[3.]]
Difference:  [[0.44600224]]
tensor([[13.8712]], grad_fn=<CopyBackwards>)
Output:  [[13.871241]]
Target:  [[17.]]
Difference:  [[-3.1287594]]
tensor([[11.0865]], grad_fn=<CopyBackwards>)
Output:  [[11.08646]]
Target:  [[15.]]
Difference:  [[-3.91354]]
tensor([[12.0378]], grad_fn=<CopyBackwards>)
Output:  [[12.037809]]
Target:  [[8.]]
Difference:  [[4.0378094]]
tensor([[8.5567]], grad_fn=<CopyBackwards>)
Output:  [[8.556719]]
Target:  [[11.]]
Difference:  [[-2.4432812]]
tensor([[10.2277]], grad_fn=<CopyBackwards>)
Output:  [[10.227651]]
Target:  [[14.]]
Difference:  [[-3.7723494]]
tensor([[11.2649]], grad_fn=<CopyBackwards>)
Output:  [[11.264933]]
Target:  [[16.]]
Difference:  [[-4.7350674]]
tensor([[5.0621]], grad_fn=<CopyBackwards>)
Output:  [[5.06207]]
Target:  [[5.]]
Difference:  [[0.062069

Output:  [[16.086573]]
Target:  [[22.]]
Difference:  [[-5.9134274]]
tensor([[8.4226]], grad_fn=<CopyBackwards>)
Output:  [[8.4226055]]
Target:  [[8.]]
Difference:  [[0.4226055]]
tensor([[6.7870]], grad_fn=<CopyBackwards>)
Output:  [[6.787049]]
Target:  [[9.]]
Difference:  [[-2.2129512]]
tensor([[6.5450]], grad_fn=<CopyBackwards>)
Output:  [[6.5450044]]
Target:  [[5.]]
Difference:  [[1.5450044]]
tensor([[3.5002]], grad_fn=<CopyBackwards>)
Output:  [[3.5001585]]
Target:  [[2.]]
Difference:  [[1.5001585]]
tensor([[14.7554]], grad_fn=<CopyBackwards>)
Output:  [[14.755417]]
Target:  [[15.]]
Difference:  [[-0.24458313]]
tensor([[5.8469]], grad_fn=<CopyBackwards>)
Output:  [[5.8469296]]
Target:  [[11.]]
Difference:  [[-5.1530704]]
tensor([[4.4758]], grad_fn=<CopyBackwards>)
Output:  [[4.4757695]]
Target:  [[4.]]
Difference:  [[0.47576952]]
tensor([[2.8847]], grad_fn=<CopyBackwards>)
Output:  [[2.8847427]]
Target:  [[3.]]
Difference:  [[-0.11525726]]
tensor([[12.3838]], grad_fn=<CopyBackwards>

[{845: {2.0: array([[2.0312984]], dtype=float32),
   3.0: array([[3.4460022]], dtype=float32),
   17.0: array([[13.871241]], dtype=float32),
   15.0: array([[11.08646]], dtype=float32),
   8.0: array([[12.037809]], dtype=float32),
   11.0: array([[8.556719]], dtype=float32),
   14.0: array([[10.227651]], dtype=float32),
   16.0: array([[11.264933]], dtype=float32),
   5.0: array([[5.06207]], dtype=float32),
   24.0: array([[22.33629]], dtype=float32),
   13.0: array([[13.723007]], dtype=float32),
   12.0: array([[10.458667]], dtype=float32),
   4.0: array([[2.572261]], dtype=float32),
   1.0: array([[1.1849891]], dtype=float32),
   10.0: array([[12.903986]], dtype=float32),
   9.0: array([[12.729527]], dtype=float32),
   7.0: array([[8.240048]], dtype=float32),
   6.0: array([[8.863543]], dtype=float32),
   18.0: array([[14.264586]], dtype=float32),
   19.0: array([[15.837524]], dtype=float32),
   20.0: array([[22.53565]], dtype=float32),
   21.0: array([[22.73823]], dtype=float32)},
 

In [127]:
split_by_race[845].head()

,raceId,year,grandprix_name,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructor_name,stop_binary,...,constructorId_205.0,constructorId_206.0,constructorId_207.0,constructorId_208.0,constructorId_209.0,constructorId_210.0,status_clean_DNF,status_clean_Finished,status_clean_lapped,driverId
0,845.0,2011.0,Spanish Grand Prix,1.0,4.0,96991.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,845.0,2011.0,Spanish Grand Prix,2.0,4.0,91189.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,845.0,2011.0,Spanish Grand Prix,3.0,4.0,90239.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,845.0,2011.0,Spanish Grand Prix,4.0,4.0,90011.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,845.0,2011.0,Spanish Grand Prix,5.0,4.0,90083.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [125]:
import operator

sorted_results = sorted(total_results[0][845].items(), key=operator.itemgetter(1))

In [126]:
sorted_results

[(1.0, array([[1.1849891]], dtype=float32)),
 (2.0, array([[2.0312984]], dtype=float32)),
 (4.0, array([[2.572261]], dtype=float32)),
 (3.0, array([[3.4460022]], dtype=float32)),
 (5.0, array([[5.06207]], dtype=float32)),
 (7.0, array([[8.240048]], dtype=float32)),
 (11.0, array([[8.556719]], dtype=float32)),
 (6.0, array([[8.863543]], dtype=float32)),
 (14.0, array([[10.227651]], dtype=float32)),
 (12.0, array([[10.458667]], dtype=float32)),
 (15.0, array([[11.08646]], dtype=float32)),
 (16.0, array([[11.264933]], dtype=float32)),
 (8.0, array([[12.037809]], dtype=float32)),
 (9.0, array([[12.729527]], dtype=float32)),
 (10.0, array([[12.903986]], dtype=float32)),
 (13.0, array([[13.723007]], dtype=float32)),
 (17.0, array([[13.871241]], dtype=float32)),
 (18.0, array([[14.264586]], dtype=float32)),
 (19.0, array([[15.837524]], dtype=float32)),
 (24.0, array([[22.33629]], dtype=float32)),
 (20.0, array([[22.53565]], dtype=float32)),
 (21.0, array([[22.73823]], dtype=float32))]

In [104]:
def test_times(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            #files.listdir(path)
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model(data).cpu()
            print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            print("Output: ", out)
            print("Target: ", target)
            help_dict[target[0][0]] = out
            print("Difference: ", out - target)
            count+=1
        result_dict[key] = help_dict
    print("Total:", total)
    return result_dict

In [108]:
test_final.keys()

dict_keys([845, 904, 911, 950, 988])

In [54]:
def test():
    total = []
    for data, target in test_data:
        model.eval()
        #files.listdir(path)
        data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0)
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        out = model(data).cpu()
        #print(out)
        out = out.detach().numpy()
        #out = np.round(out)
        target = target.cpu()
        target = target.detach().numpy()
        #print(data)
        #print(data["driverId"])
        if out <=41 :
            total.append(out)
            #total += abs(out - target[0][0])
            print("current_position: ", data[0][0].item())
            print("Output: ", out)
            print("Target: ", target)
            print("Difference: ", out - target)
            print("============================")
    print("Total:", total)
    return total


## Test with times

In [53]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        self.fc1 = nn.Linear(110, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 190)
        self.fc4 = nn.Linear(190, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        x = self.dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        #x = self.dropout(x)
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x
        

In [54]:

model = Netz()
model.cuda()
        

optimizer = optim.Adam(model.parameters(), lr = 0.0001)        
def train(epoch):
    model.train()
    batch_id = 0
    for data, target in train_data:
        data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0).cuda()
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        optimizer.zero_grad()
        out = model(data)
        #print("Out: ", out, out.size())
        #print("Target: ", target, target.size())
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_id *len(data), len(train_data),
        100. * batch_id / len(train_data), loss.item()))
        batch_id +=1
        

In [55]:
for epoch in range(1,7):
    train(epoch)

Train Epoche: 1 [0/2883 (0%)]	Loss: 1.615328
Train Epoche: 1 [1/2883 (0%)]	Loss: 0.102484
Train Epoche: 1 [2/2883 (0%)]	Loss: 3.009801
Train Epoche: 1 [3/2883 (0%)]	Loss: 1.942770
Train Epoche: 1 [4/2883 (0%)]	Loss: 2.055624
Train Epoche: 1 [5/2883 (0%)]	Loss: 1.987560
Train Epoche: 1 [6/2883 (0%)]	Loss: 0.012229
Train Epoche: 1 [7/2883 (0%)]	Loss: 4.392323
Train Epoche: 1 [8/2883 (0%)]	Loss: 2.013796
Train Epoche: 1 [9/2883 (0%)]	Loss: 2.652034
Train Epoche: 1 [10/2883 (0%)]	Loss: 0.147992
Train Epoche: 1 [11/2883 (0%)]	Loss: 2.862522
Train Epoche: 1 [12/2883 (0%)]	Loss: 2.196306
Train Epoche: 1 [13/2883 (0%)]	Loss: 1.999219
Train Epoche: 1 [14/2883 (0%)]	Loss: 2.021939
Train Epoche: 1 [15/2883 (1%)]	Loss: 1.473375
Train Epoche: 1 [16/2883 (1%)]	Loss: 3.443859
Train Epoche: 1 [17/2883 (1%)]	Loss: 2.701451
Train Epoche: 1 [18/2883 (1%)]	Loss: 1.907394
Train Epoche: 1 [19/2883 (1%)]	Loss: 2.159170
Train Epoche: 1 [20/2883 (1%)]	Loss: 2.338283
Train Epoche: 1 [21/2883 (1%)]	Loss: 1.97822

Train Epoche: 1 [348/2883 (12%)]	Loss: 0.053770
Train Epoche: 1 [349/2883 (12%)]	Loss: 0.115331
Train Epoche: 1 [350/2883 (12%)]	Loss: 0.160013
Train Epoche: 1 [351/2883 (12%)]	Loss: 0.597630
Train Epoche: 1 [352/2883 (12%)]	Loss: 0.333456
Train Epoche: 1 [353/2883 (12%)]	Loss: 0.000654
Train Epoche: 1 [354/2883 (12%)]	Loss: 0.041915
Train Epoche: 1 [355/2883 (12%)]	Loss: 0.105968
Train Epoche: 1 [356/2883 (12%)]	Loss: 0.817232
Train Epoche: 1 [357/2883 (12%)]	Loss: 0.000188
Train Epoche: 1 [358/2883 (12%)]	Loss: 0.025687
Train Epoche: 1 [359/2883 (12%)]	Loss: 1.045898
Train Epoche: 1 [360/2883 (12%)]	Loss: 0.011403
Train Epoche: 1 [361/2883 (13%)]	Loss: 0.006512
Train Epoche: 1 [362/2883 (13%)]	Loss: 0.004644
Train Epoche: 1 [363/2883 (13%)]	Loss: 0.120873
Train Epoche: 1 [364/2883 (13%)]	Loss: 0.794421
Train Epoche: 1 [365/2883 (13%)]	Loss: 0.220641
Train Epoche: 1 [366/2883 (13%)]	Loss: 0.191187
Train Epoche: 1 [367/2883 (13%)]	Loss: 0.056428
Train Epoche: 1 [368/2883 (13%)]	Loss: 0

Train Epoche: 1 [688/2883 (24%)]	Loss: 0.024939
Train Epoche: 1 [689/2883 (24%)]	Loss: 0.036380
Train Epoche: 1 [690/2883 (24%)]	Loss: 0.001969
Train Epoche: 1 [691/2883 (24%)]	Loss: 0.058197
Train Epoche: 1 [692/2883 (24%)]	Loss: 0.054254
Train Epoche: 1 [693/2883 (24%)]	Loss: 0.029721
Train Epoche: 1 [694/2883 (24%)]	Loss: 0.106078
Train Epoche: 1 [695/2883 (24%)]	Loss: 0.412847
Train Epoche: 1 [696/2883 (24%)]	Loss: 0.145416
Train Epoche: 1 [697/2883 (24%)]	Loss: 0.003486
Train Epoche: 1 [698/2883 (24%)]	Loss: 0.667946
Train Epoche: 1 [699/2883 (24%)]	Loss: 0.025020
Train Epoche: 1 [700/2883 (24%)]	Loss: 0.000035
Train Epoche: 1 [701/2883 (24%)]	Loss: 0.013601
Train Epoche: 1 [702/2883 (24%)]	Loss: 0.006162
Train Epoche: 1 [703/2883 (24%)]	Loss: 0.011257
Train Epoche: 1 [704/2883 (24%)]	Loss: 0.009504
Train Epoche: 1 [705/2883 (24%)]	Loss: 0.285423
Train Epoche: 1 [706/2883 (24%)]	Loss: 0.003635
Train Epoche: 1 [707/2883 (25%)]	Loss: 0.892681
Train Epoche: 1 [708/2883 (25%)]	Loss: 0

Train Epoche: 1 [1028/2883 (36%)]	Loss: 0.100965
Train Epoche: 1 [1029/2883 (36%)]	Loss: 0.166807
Train Epoche: 1 [1030/2883 (36%)]	Loss: 0.005136
Train Epoche: 1 [1031/2883 (36%)]	Loss: 0.470149
Train Epoche: 1 [1032/2883 (36%)]	Loss: 0.012165
Train Epoche: 1 [1033/2883 (36%)]	Loss: 0.049641
Train Epoche: 1 [1034/2883 (36%)]	Loss: 0.744962
Train Epoche: 1 [1035/2883 (36%)]	Loss: 0.128888
Train Epoche: 1 [1036/2883 (36%)]	Loss: 0.123697
Train Epoche: 1 [1037/2883 (36%)]	Loss: 0.014287
Train Epoche: 1 [1038/2883 (36%)]	Loss: 1.243447
Train Epoche: 1 [1039/2883 (36%)]	Loss: 0.020306
Train Epoche: 1 [1040/2883 (36%)]	Loss: 0.032952
Train Epoche: 1 [1041/2883 (36%)]	Loss: 0.002529
Train Epoche: 1 [1042/2883 (36%)]	Loss: 0.040370
Train Epoche: 1 [1043/2883 (36%)]	Loss: 0.000001
Train Epoche: 1 [1044/2883 (36%)]	Loss: 0.266043
Train Epoche: 1 [1045/2883 (36%)]	Loss: 0.012603
Train Epoche: 1 [1046/2883 (36%)]	Loss: 0.472611
Train Epoche: 1 [1047/2883 (36%)]	Loss: 0.006040
Train Epoche: 1 [104

Train Epoche: 1 [1362/2883 (47%)]	Loss: 0.017512
Train Epoche: 1 [1363/2883 (47%)]	Loss: 0.062048
Train Epoche: 1 [1364/2883 (47%)]	Loss: 0.157193
Train Epoche: 1 [1365/2883 (47%)]	Loss: 0.154591
Train Epoche: 1 [1366/2883 (47%)]	Loss: 0.262209
Train Epoche: 1 [1367/2883 (47%)]	Loss: 0.066491
Train Epoche: 1 [1368/2883 (47%)]	Loss: 0.025176
Train Epoche: 1 [1369/2883 (47%)]	Loss: 0.020784
Train Epoche: 1 [1370/2883 (48%)]	Loss: 0.205242
Train Epoche: 1 [1371/2883 (48%)]	Loss: 0.219855
Train Epoche: 1 [1372/2883 (48%)]	Loss: 0.076441
Train Epoche: 1 [1373/2883 (48%)]	Loss: 0.055344
Train Epoche: 1 [1374/2883 (48%)]	Loss: 0.078774
Train Epoche: 1 [1375/2883 (48%)]	Loss: 0.058244
Train Epoche: 1 [1376/2883 (48%)]	Loss: 0.000386
Train Epoche: 1 [1377/2883 (48%)]	Loss: 0.358626
Train Epoche: 1 [1378/2883 (48%)]	Loss: 0.019916
Train Epoche: 1 [1379/2883 (48%)]	Loss: 0.031001
Train Epoche: 1 [1380/2883 (48%)]	Loss: 0.000007
Train Epoche: 1 [1381/2883 (48%)]	Loss: 0.309784
Train Epoche: 1 [138

...

Train Epoche: 6 [1437/2883 (50%)]	Loss: 0.002365
Train Epoche: 6 [1438/2883 (50%)]	Loss: 0.109983
Train Epoche: 6 [1439/2883 (50%)]	Loss: 0.000105
Train Epoche: 6 [1440/2883 (50%)]	Loss: 0.007098
Train Epoche: 6 [1441/2883 (50%)]	Loss: 0.001609
Train Epoche: 6 [1442/2883 (50%)]	Loss: 0.028376
Train Epoche: 6 [1443/2883 (50%)]	Loss: 0.008153
Train Epoche: 6 [1444/2883 (50%)]	Loss: 0.169230
Train Epoche: 6 [1445/2883 (50%)]	Loss: 0.014656
Train Epoche: 6 [1446/2883 (50%)]	Loss: 0.002843
Train Epoche: 6 [1447/2883 (50%)]	Loss: 0.056150
Train Epoche: 6 [1448/2883 (50%)]	Loss: 0.183955
Train Epoche: 6 [1449/2883 (50%)]	Loss: 0.014796
Train Epoche: 6 [1450/2883 (50%)]	Loss: 0.023682
Train Epoche: 6 [1451/2883 (50%)]	Loss: 0.018435
Train Epoche: 6 [1452/2883 (50%)]	Loss: 0.001367
Train Epoche: 6 [1453/2883 (50%)]	Loss: 0.004793
Train Epoche: 6 [1454/2883 (50%)]	Loss: 0.014314
Train Epoche: 6 [1455/2883 (50%)]	Loss: 0.000664
Train Epoche: 6 [1456/2883 (51%)]	Loss: 0.043213
Train Epoche: 6 [145

Train Epoche: 6 [1771/2883 (61%)]	Loss: 0.157999
Train Epoche: 6 [1772/2883 (61%)]	Loss: 0.049229
Train Epoche: 6 [1773/2883 (61%)]	Loss: 0.039074
Train Epoche: 6 [1774/2883 (62%)]	Loss: 0.005917
Train Epoche: 6 [1775/2883 (62%)]	Loss: 0.005273
Train Epoche: 6 [1776/2883 (62%)]	Loss: 0.190854
Train Epoche: 6 [1777/2883 (62%)]	Loss: 0.012160
Train Epoche: 6 [1778/2883 (62%)]	Loss: 0.044296
Train Epoche: 6 [1779/2883 (62%)]	Loss: 0.011445
Train Epoche: 6 [1780/2883 (62%)]	Loss: 0.006875
Train Epoche: 6 [1781/2883 (62%)]	Loss: 0.031440
Train Epoche: 6 [1782/2883 (62%)]	Loss: 0.215358
Train Epoche: 6 [1783/2883 (62%)]	Loss: 0.015832
Train Epoche: 6 [1784/2883 (62%)]	Loss: 0.004255
Train Epoche: 6 [1785/2883 (62%)]	Loss: 0.015890
Train Epoche: 6 [1786/2883 (62%)]	Loss: 0.025333
Train Epoche: 6 [1787/2883 (62%)]	Loss: 0.002508
Train Epoche: 6 [1788/2883 (62%)]	Loss: 0.044509
Train Epoche: 6 [1789/2883 (62%)]	Loss: 0.028004
Train Epoche: 6 [1790/2883 (62%)]	Loss: 0.004086
Train Epoche: 6 [179

Train Epoche: 6 [2105/2883 (73%)]	Loss: 0.003746
Train Epoche: 6 [2106/2883 (73%)]	Loss: 0.027914
Train Epoche: 6 [2107/2883 (73%)]	Loss: 0.001540
Train Epoche: 6 [2108/2883 (73%)]	Loss: 0.013641
Train Epoche: 6 [2109/2883 (73%)]	Loss: 0.011897
Train Epoche: 6 [2110/2883 (73%)]	Loss: 0.007835
Train Epoche: 6 [2111/2883 (73%)]	Loss: 0.003771
Train Epoche: 6 [2112/2883 (73%)]	Loss: 0.000133
Train Epoche: 6 [2113/2883 (73%)]	Loss: 0.006557
Train Epoche: 6 [2114/2883 (73%)]	Loss: 0.013575
Train Epoche: 6 [2115/2883 (73%)]	Loss: 0.000053
Train Epoche: 6 [2116/2883 (73%)]	Loss: 0.002133
Train Epoche: 6 [2117/2883 (73%)]	Loss: 0.001070
Train Epoche: 6 [2118/2883 (73%)]	Loss: 0.025020
Train Epoche: 6 [2119/2883 (73%)]	Loss: 0.089429
Train Epoche: 6 [2120/2883 (74%)]	Loss: 0.000082
Train Epoche: 6 [2121/2883 (74%)]	Loss: 0.004351
Train Epoche: 6 [2122/2883 (74%)]	Loss: 0.017641
Train Epoche: 6 [2123/2883 (74%)]	Loss: 0.230073
Train Epoche: 6 [2124/2883 (74%)]	Loss: 0.002670
Train Epoche: 6 [212

Train Epoche: 6 [2439/2883 (85%)]	Loss: 0.005639
Train Epoche: 6 [2440/2883 (85%)]	Loss: 0.058850
Train Epoche: 6 [2441/2883 (85%)]	Loss: 0.005254
Train Epoche: 6 [2442/2883 (85%)]	Loss: 0.004254
Train Epoche: 6 [2443/2883 (85%)]	Loss: 0.096081
Train Epoche: 6 [2444/2883 (85%)]	Loss: 0.000127
Train Epoche: 6 [2445/2883 (85%)]	Loss: 0.017645
Train Epoche: 6 [2446/2883 (85%)]	Loss: 0.033958
Train Epoche: 6 [2447/2883 (85%)]	Loss: 0.040974
Train Epoche: 6 [2448/2883 (85%)]	Loss: 0.000988
Train Epoche: 6 [2449/2883 (85%)]	Loss: 0.007560
Train Epoche: 6 [2450/2883 (85%)]	Loss: 0.091211
Train Epoche: 6 [2451/2883 (85%)]	Loss: 0.049951
Train Epoche: 6 [2452/2883 (85%)]	Loss: 0.000158
Train Epoche: 6 [2453/2883 (85%)]	Loss: 0.079082
Train Epoche: 6 [2454/2883 (85%)]	Loss: 0.000678
Train Epoche: 6 [2455/2883 (85%)]	Loss: 0.124229
Train Epoche: 6 [2456/2883 (85%)]	Loss: 0.055142
Train Epoche: 6 [2457/2883 (85%)]	Loss: 0.001511
Train Epoche: 6 [2458/2883 (85%)]	Loss: 0.004125
Train Epoche: 6 [245

Train Epoche: 6 [2773/2883 (96%)]	Loss: 0.008789
Train Epoche: 6 [2774/2883 (96%)]	Loss: 0.012191
Train Epoche: 6 [2775/2883 (96%)]	Loss: 0.000663
Train Epoche: 6 [2776/2883 (96%)]	Loss: 0.011591
Train Epoche: 6 [2777/2883 (96%)]	Loss: 0.018299
Train Epoche: 6 [2778/2883 (96%)]	Loss: 0.000530
Train Epoche: 6 [2779/2883 (96%)]	Loss: 0.022906
Train Epoche: 6 [2780/2883 (96%)]	Loss: 0.011801
Train Epoche: 6 [2781/2883 (96%)]	Loss: 0.007912
Train Epoche: 6 [2782/2883 (96%)]	Loss: 0.001466
Train Epoche: 6 [2783/2883 (97%)]	Loss: 0.155559
Train Epoche: 6 [2784/2883 (97%)]	Loss: 0.001659
Train Epoche: 6 [2785/2883 (97%)]	Loss: 0.059248
Train Epoche: 6 [2786/2883 (97%)]	Loss: 0.093783
Train Epoche: 6 [2787/2883 (97%)]	Loss: 0.048588
Train Epoche: 6 [2788/2883 (97%)]	Loss: 0.035056
Train Epoche: 6 [2789/2883 (97%)]	Loss: 0.014625
Train Epoche: 6 [2790/2883 (97%)]	Loss: 0.069798
Train Epoche: 6 [2791/2883 (97%)]	Loss: 0.000025
Train Epoche: 6 [2792/2883 (97%)]	Loss: 0.005059
Train Epoche: 6 [279

In [56]:
result_dict = test_times()

tensor([[1.2837]], grad_fn=<CopyBackwards>)
Difference:  [[-0.31926084]]
tensor([[1.2735]], grad_fn=<CopyBackwards>)
Difference:  [[-0.34694338]]
tensor([[1.2756]], grad_fn=<CopyBackwards>)
Difference:  [[-0.3283006]]
tensor([[1.3227]], grad_fn=<CopyBackwards>)
Difference:  [[-0.30255592]]
tensor([[1.3438]], grad_fn=<CopyBackwards>)
Difference:  [[-0.25054443]]
tensor([[1.0525]], grad_fn=<CopyBackwards>)
Difference:  [[0.14678383]]
tensor([[1.2786]], grad_fn=<CopyBackwards>)
Difference:  [[-0.34367]]
tensor([[1.2398]], grad_fn=<CopyBackwards>)
Difference:  [[-0.38193464]]
tensor([[1.2548]], grad_fn=<CopyBackwards>)
Difference:  [[-0.32841027]]
tensor([[1.2766]], grad_fn=<CopyBackwards>)
Difference:  [[-0.29935277]]
tensor([[1.3232]], grad_fn=<CopyBackwards>)
Difference:  [[-0.27292764]]
tensor([[1.3229]], grad_fn=<CopyBackwards>)
Difference:  [[-0.27516162]]
tensor([[0.7461]], grad_fn=<CopyBackwards>)
Difference:  [[0.16438007]]
tensor([[1.2968]], grad_fn=<CopyBackwards>)
Difference:  

In [57]:
result_dict

{9.0: array([[1.2837045]], dtype=float32),
 12.0: array([[1.2735454]], dtype=float32),
 10.0: array([[1.2756276]], dtype=float32),
 18.0: array([[1.3227462]], dtype=float32),
 6.0: array([[1.3438375]], dtype=float32),
 20.0: array([[0.7460873]], dtype=float32),
 16.0: array([[1.2785926]], dtype=float32),
 15.0: array([[1.2397785]], dtype=float32),
 4.0: array([[1.2547697]], dtype=float32),
 3.0: array([[1.2765895]], dtype=float32),
 7.0: array([[1.3232179]], dtype=float32),
 8.0: array([[1.3228865]], dtype=float32),
 5.0: array([[1.2968102]], dtype=float32),
 17.0: array([[1.3031565]], dtype=float32),
 14.0: array([[1.2890464]], dtype=float32),
 2.0: array([[1.2784088]], dtype=float32),
 1.0: array([[1.2735944]], dtype=float32),
 11.0: array([[1.3023514]], dtype=float32),
 13.0: array([[1.2926636]], dtype=float32)}

In [58]:
import operator

sorted_results = sorted(result_dict.items(), key=operator.itemgetter(1))

In [59]:
sorted_results

[(20.0, array([[0.7460873]], dtype=float32)),
 (15.0, array([[1.2397785]], dtype=float32)),
 (4.0, array([[1.2547697]], dtype=float32)),
 (12.0, array([[1.2735454]], dtype=float32)),
 (1.0, array([[1.2735944]], dtype=float32)),
 (10.0, array([[1.2756276]], dtype=float32)),
 (3.0, array([[1.2765895]], dtype=float32)),
 (2.0, array([[1.2784088]], dtype=float32)),
 (16.0, array([[1.2785926]], dtype=float32)),
 (9.0, array([[1.2837045]], dtype=float32)),
 (14.0, array([[1.2890464]], dtype=float32)),
 (13.0, array([[1.2926636]], dtype=float32)),
 (5.0, array([[1.2968102]], dtype=float32)),
 (11.0, array([[1.3023514]], dtype=float32)),
 (17.0, array([[1.3031565]], dtype=float32)),
 (18.0, array([[1.3227462]], dtype=float32)),
 (8.0, array([[1.3228865]], dtype=float32)),
 (7.0, array([[1.3232179]], dtype=float32)),
 (6.0, array([[1.3438375]], dtype=float32))]

In [40]:
def test_times():
    total = 0
    count = 0
    result_dict = {}
    for data, target in test_data:
        model.eval()
        #files.listdir(path)
        data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0)
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        out = model(data).cpu()
        print(out)
        out = out.detach().numpy()
        #out = np.round(out)
        target = target.cpu()
        target = target.detach().numpy()
        #print(data)
        #print(data["driverId"])
        total += abs(out - target[0][0])
        #print("current_position: ", data[0][0].item())
        #print("Output: ", out)
        #print("Target: ", target)
        result_dict[temp_y_podium[count]] = out
        print("Difference: ", out - target)
        count+=1
    print("Total:", total)
    return result_dict

# Das Funktioniert schonmal Super

In [ ]:
train_data = []
nogo_columns_hannah = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary','constructorId_1.0',
                 'constructorId_3.0',
                 'constructorId_4.0',
                 'constructorId_5.0',
                 'constructorId_6.0',
                 'constructorId_9.0',
                 'constructorId_10.0',
                 'constructorId_15.0',
                 'constructorId_131.0',
                 'constructorId_164.0',
                 'constructorId_166.0',
                 'constructorId_205.0',
                 'constructorId_206.0',
                 'constructorId_207.0',
                 'constructorId_208.0',
                 'constructorId_209.0',
                 'constructorId_210.0']
nogo_columns = ['year', 'podium_position', 'raceId','lap_number','total_laps','driverId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds']
for key, value in sliced_races.items():
    for did in value.driverId.unique():
        temp = value.where(value.driverId == did).dropna(how = "all")
        if list(temp["podium_position"])[0] > 30: #Top x finish positions
            pp = 1
        else:
            
            temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
            temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
            #temp_y = temp_y[0]
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
            temp_x = temp_x.tail(1)
            temp_x['stop_binary'] = stops
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            #temp_x = x_tensor.float()
            train_data.append((x_tensor, [temp_y[0]]))
random.shuffle(train_data)
test_data = train_data[len(train_data)-20:]
train_data = train_data[0:len(train_data)-20]


In [47]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        self.fc1 = nn.Linear(110, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 150)
        self.fc4 = nn.Linear(150, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        #self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        #x = self.conv_dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x
        

In [57]:

model = Netz()
model.cuda()
        

optimizer = optim.Adam(model.parameters(), lr = 0.0001)        
def train(epoch):
    model.train()
    batch_id = 0
    for data, target in train_data:
        data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0)
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        optimizer.zero_grad()
        out = model(data)
        #print("Out: ", out, out.size())
        #print("Target: ", target, target.size())
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_id *len(data), len(train_data),
        100. * batch_id / len(train_data), loss.item()))
        batch_id +=1
        
    random.shuffle(train_data)
        

In [62]:
for epoch in range(1,15):
    train(epoch)

Train Epoche: 1 [0/1269 (0%)]	Loss: 1.599254
Train Epoche: 1 [1/1269 (0%)]	Loss: 15.635448
Train Epoche: 1 [2/1269 (0%)]	Loss: 0.647313
Train Epoche: 1 [3/1269 (0%)]	Loss: 14.971535
Train Epoche: 1 [4/1269 (0%)]	Loss: 0.134667
Train Epoche: 1 [5/1269 (0%)]	Loss: 0.244088
Train Epoche: 1 [6/1269 (0%)]	Loss: 4.045896
Train Epoche: 1 [7/1269 (1%)]	Loss: 4.100158
Train Epoche: 1 [8/1269 (1%)]	Loss: 0.537043
Train Epoche: 1 [9/1269 (1%)]	Loss: 0.581020
Train Epoche: 1 [10/1269 (1%)]	Loss: 0.004266
Train Epoche: 1 [11/1269 (1%)]	Loss: 0.000588
Train Epoche: 1 [12/1269 (1%)]	Loss: 0.045169
Train Epoche: 1 [13/1269 (1%)]	Loss: 0.003935
Train Epoche: 1 [14/1269 (1%)]	Loss: 7.399226
Train Epoche: 1 [15/1269 (1%)]	Loss: 0.157109
Train Epoche: 1 [16/1269 (1%)]	Loss: 1.479398
Train Epoche: 1 [17/1269 (1%)]	Loss: 0.010022
Train Epoche: 1 [18/1269 (1%)]	Loss: 5.809720
Train Epoche: 1 [19/1269 (1%)]	Loss: 1.877080
Train Epoche: 1 [20/1269 (2%)]	Loss: 0.797147
Train Epoche: 1 [21/1269 (2%)]	Loss: 0.546

Train Epoche: 1 [345/1269 (27%)]	Loss: 1.193618
Train Epoche: 1 [346/1269 (27%)]	Loss: 0.002614
Train Epoche: 1 [347/1269 (27%)]	Loss: 3.444336
Train Epoche: 1 [348/1269 (27%)]	Loss: 2.747650
Train Epoche: 1 [349/1269 (28%)]	Loss: 0.025340
Train Epoche: 1 [350/1269 (28%)]	Loss: 0.502342
Train Epoche: 1 [351/1269 (28%)]	Loss: 0.814201
Train Epoche: 1 [352/1269 (28%)]	Loss: 0.302681
Train Epoche: 1 [353/1269 (28%)]	Loss: 0.161426
Train Epoche: 1 [354/1269 (28%)]	Loss: 0.060711
Train Epoche: 1 [355/1269 (28%)]	Loss: 5.014762
Train Epoche: 1 [356/1269 (28%)]	Loss: 0.697145
Train Epoche: 1 [357/1269 (28%)]	Loss: 0.000717
Train Epoche: 1 [358/1269 (28%)]	Loss: 7.145361
Train Epoche: 1 [359/1269 (28%)]	Loss: 6.156019
Train Epoche: 1 [360/1269 (28%)]	Loss: 0.641783
Train Epoche: 1 [361/1269 (28%)]	Loss: 0.000014
Train Epoche: 1 [362/1269 (29%)]	Loss: 6.103500
Train Epoche: 1 [363/1269 (29%)]	Loss: 9.821919
Train Epoche: 1 [364/1269 (29%)]	Loss: 0.506298
Train Epoche: 1 [365/1269 (29%)]	Loss: 3

Train Epoche: 1 [685/1269 (54%)]	Loss: 0.042620
Train Epoche: 1 [686/1269 (54%)]	Loss: 0.696369
Train Epoche: 1 [687/1269 (54%)]	Loss: 0.132484
Train Epoche: 1 [688/1269 (54%)]	Loss: 1.220213
Train Epoche: 1 [689/1269 (54%)]	Loss: 0.756016
Train Epoche: 1 [690/1269 (54%)]	Loss: 0.020290
Train Epoche: 1 [691/1269 (54%)]	Loss: 36.203960
Train Epoche: 1 [692/1269 (55%)]	Loss: 4.131472
Train Epoche: 1 [693/1269 (55%)]	Loss: 1.392356
Train Epoche: 1 [694/1269 (55%)]	Loss: 0.103871
Train Epoche: 1 [695/1269 (55%)]	Loss: 1.605966
Train Epoche: 1 [696/1269 (55%)]	Loss: 3.004810
Train Epoche: 1 [697/1269 (55%)]	Loss: 0.181882
Train Epoche: 1 [698/1269 (55%)]	Loss: 0.229680
Train Epoche: 1 [699/1269 (55%)]	Loss: 1.098691
Train Epoche: 1 [700/1269 (55%)]	Loss: 0.176091
Train Epoche: 1 [701/1269 (55%)]	Loss: 0.831375
Train Epoche: 1 [702/1269 (55%)]	Loss: 0.209161
Train Epoche: 1 [703/1269 (55%)]	Loss: 1.527219
Train Epoche: 1 [704/1269 (55%)]	Loss: 0.295874
Train Epoche: 1 [705/1269 (56%)]	Loss: 

Train Epoche: 1 [1024/1269 (81%)]	Loss: 0.646417
Train Epoche: 1 [1025/1269 (81%)]	Loss: 0.070179
Train Epoche: 1 [1026/1269 (81%)]	Loss: 0.672912
Train Epoche: 1 [1027/1269 (81%)]	Loss: 0.575150
Train Epoche: 1 [1028/1269 (81%)]	Loss: 0.016174
Train Epoche: 1 [1029/1269 (81%)]	Loss: 14.775875
Train Epoche: 1 [1030/1269 (81%)]	Loss: 1.333793
Train Epoche: 1 [1031/1269 (81%)]	Loss: 1.693810
Train Epoche: 1 [1032/1269 (81%)]	Loss: 0.077847
Train Epoche: 1 [1033/1269 (81%)]	Loss: 0.688674
Train Epoche: 1 [1034/1269 (81%)]	Loss: 0.090701
Train Epoche: 1 [1035/1269 (82%)]	Loss: 0.013962
Train Epoche: 1 [1036/1269 (82%)]	Loss: 0.042376
Train Epoche: 1 [1037/1269 (82%)]	Loss: 13.764646
Train Epoche: 1 [1038/1269 (82%)]	Loss: 0.070602
Train Epoche: 1 [1039/1269 (82%)]	Loss: 0.103438
Train Epoche: 1 [1040/1269 (82%)]	Loss: 4.066104
Train Epoche: 1 [1041/1269 (82%)]	Loss: 0.183613
Train Epoche: 1 [1042/1269 (82%)]	Loss: 22.755661
Train Epoche: 1 [1043/1269 (82%)]	Loss: 1.614464
Train Epoche: 1 [

Train Epoche: 2 [93/1269 (7%)]	Loss: 4.651538
Train Epoche: 2 [94/1269 (7%)]	Loss: 0.000090
Train Epoche: 2 [95/1269 (7%)]	Loss: 0.374623
Train Epoche: 2 [96/1269 (8%)]	Loss: 4.070606
Train Epoche: 2 [97/1269 (8%)]	Loss: 0.603212
Train Epoche: 2 [98/1269 (8%)]	Loss: 3.586720
Train Epoche: 2 [99/1269 (8%)]	Loss: 29.638893
Train Epoche: 2 [100/1269 (8%)]	Loss: 0.281191
Train Epoche: 2 [101/1269 (8%)]	Loss: 0.298426
Train Epoche: 2 [102/1269 (8%)]	Loss: 0.108689
Train Epoche: 2 [103/1269 (8%)]	Loss: 0.095301
Train Epoche: 2 [104/1269 (8%)]	Loss: 0.772130
Train Epoche: 2 [105/1269 (8%)]	Loss: 1.362050
Train Epoche: 2 [106/1269 (8%)]	Loss: 0.172875
Train Epoche: 2 [107/1269 (8%)]	Loss: 2.613940
Train Epoche: 2 [108/1269 (9%)]	Loss: 0.138999
Train Epoche: 2 [109/1269 (9%)]	Loss: 0.050374
Train Epoche: 2 [110/1269 (9%)]	Loss: 2.125948
Train Epoche: 2 [111/1269 (9%)]	Loss: 0.001216
Train Epoche: 2 [112/1269 (9%)]	Loss: 4.042292
Train Epoche: 2 [113/1269 (9%)]	Loss: 0.196323
Train Epoche: 2 [11

...

Train Epoche: 13 [990/1269 (78%)]	Loss: 0.228339
Train Epoche: 13 [991/1269 (78%)]	Loss: 1.921482
Train Epoche: 13 [992/1269 (78%)]	Loss: 1.159111
Train Epoche: 13 [993/1269 (78%)]	Loss: 0.400674
Train Epoche: 13 [994/1269 (78%)]	Loss: 0.125352
Train Epoche: 13 [995/1269 (78%)]	Loss: 0.259412
Train Epoche: 13 [996/1269 (78%)]	Loss: 6.065668
Train Epoche: 13 [997/1269 (79%)]	Loss: 3.350408
Train Epoche: 13 [998/1269 (79%)]	Loss: 1.978340
Train Epoche: 13 [999/1269 (79%)]	Loss: 0.002046
Train Epoche: 13 [1000/1269 (79%)]	Loss: 0.785824
Train Epoche: 13 [1001/1269 (79%)]	Loss: 0.019936
Train Epoche: 13 [1002/1269 (79%)]	Loss: 0.181124
Train Epoche: 13 [1003/1269 (79%)]	Loss: 0.860025
Train Epoche: 13 [1004/1269 (79%)]	Loss: 4.350783
Train Epoche: 13 [1005/1269 (79%)]	Loss: 2.088722
Train Epoche: 13 [1006/1269 (79%)]	Loss: 0.005048
Train Epoche: 13 [1007/1269 (79%)]	Loss: 2.035751
Train Epoche: 13 [1008/1269 (79%)]	Loss: 0.556067
Train Epoche: 13 [1009/1269 (80%)]	Loss: 4.130439
Train Epoc

Train Epoche: 14 [50/1269 (4%)]	Loss: 0.059386
Train Epoche: 14 [51/1269 (4%)]	Loss: 11.757560
Train Epoche: 14 [52/1269 (4%)]	Loss: 0.044472
Train Epoche: 14 [53/1269 (4%)]	Loss: 1.987296
Train Epoche: 14 [54/1269 (4%)]	Loss: 1.711293
Train Epoche: 14 [55/1269 (4%)]	Loss: 0.706202
Train Epoche: 14 [56/1269 (4%)]	Loss: 0.918853
Train Epoche: 14 [57/1269 (4%)]	Loss: 0.027286
Train Epoche: 14 [58/1269 (5%)]	Loss: 0.001679
Train Epoche: 14 [59/1269 (5%)]	Loss: 0.050323
Train Epoche: 14 [60/1269 (5%)]	Loss: 0.070879
Train Epoche: 14 [61/1269 (5%)]	Loss: 1.237680
Train Epoche: 14 [62/1269 (5%)]	Loss: 2.777566
Train Epoche: 14 [63/1269 (5%)]	Loss: 0.004402
Train Epoche: 14 [64/1269 (5%)]	Loss: 0.286528
Train Epoche: 14 [65/1269 (5%)]	Loss: 0.089406
Train Epoche: 14 [66/1269 (5%)]	Loss: 8.965303
Train Epoche: 14 [67/1269 (5%)]	Loss: 0.715999
Train Epoche: 14 [68/1269 (5%)]	Loss: 2.025071
Train Epoche: 14 [69/1269 (5%)]	Loss: 7.061570
Train Epoche: 14 [70/1269 (6%)]	Loss: 1.146605
Train Epoche

Train Epoche: 14 [385/1269 (30%)]	Loss: 0.561794
Train Epoche: 14 [386/1269 (30%)]	Loss: 9.617791
Train Epoche: 14 [387/1269 (30%)]	Loss: 12.694172
Train Epoche: 14 [388/1269 (31%)]	Loss: 0.323143
Train Epoche: 14 [389/1269 (31%)]	Loss: 0.241809
Train Epoche: 14 [390/1269 (31%)]	Loss: 0.357878
Train Epoche: 14 [391/1269 (31%)]	Loss: 0.046480
Train Epoche: 14 [392/1269 (31%)]	Loss: 2.012095
Train Epoche: 14 [393/1269 (31%)]	Loss: 5.149759
Train Epoche: 14 [394/1269 (31%)]	Loss: 0.334759
Train Epoche: 14 [395/1269 (31%)]	Loss: 0.158477
Train Epoche: 14 [396/1269 (31%)]	Loss: 0.655277
Train Epoche: 14 [397/1269 (31%)]	Loss: 0.073459
Train Epoche: 14 [398/1269 (31%)]	Loss: 1.338257
Train Epoche: 14 [399/1269 (31%)]	Loss: 0.481421
Train Epoche: 14 [400/1269 (32%)]	Loss: 4.403715
Train Epoche: 14 [401/1269 (32%)]	Loss: 3.956566
Train Epoche: 14 [402/1269 (32%)]	Loss: 0.550118
Train Epoche: 14 [403/1269 (32%)]	Loss: 1.861652
Train Epoche: 14 [404/1269 (32%)]	Loss: 1.270718
Train Epoche: 14 [4

Train Epoche: 14 [719/1269 (57%)]	Loss: 0.051316
Train Epoche: 14 [720/1269 (57%)]	Loss: 0.865513
Train Epoche: 14 [721/1269 (57%)]	Loss: 12.639750
Train Epoche: 14 [722/1269 (57%)]	Loss: 0.213011
Train Epoche: 14 [723/1269 (57%)]	Loss: 1.895263
Train Epoche: 14 [724/1269 (57%)]	Loss: 0.318542
Train Epoche: 14 [725/1269 (57%)]	Loss: 0.429087
Train Epoche: 14 [726/1269 (57%)]	Loss: 0.135192
Train Epoche: 14 [727/1269 (57%)]	Loss: 0.236915
Train Epoche: 14 [728/1269 (57%)]	Loss: 0.167500
Train Epoche: 14 [729/1269 (57%)]	Loss: 0.002227
Train Epoche: 14 [730/1269 (58%)]	Loss: 0.016822
Train Epoche: 14 [731/1269 (58%)]	Loss: 0.325790
Train Epoche: 14 [732/1269 (58%)]	Loss: 2.670470
Train Epoche: 14 [733/1269 (58%)]	Loss: 0.207715
Train Epoche: 14 [734/1269 (58%)]	Loss: 1.161490
Train Epoche: 14 [735/1269 (58%)]	Loss: 2.681457
Train Epoche: 14 [736/1269 (58%)]	Loss: 3.439072
Train Epoche: 14 [737/1269 (58%)]	Loss: 7.353583
Train Epoche: 14 [738/1269 (58%)]	Loss: 0.734056
Train Epoche: 14 [7

Train Epoche: 14 [1051/1269 (83%)]	Loss: 0.080489
Train Epoche: 14 [1052/1269 (83%)]	Loss: 2.274495
Train Epoche: 14 [1053/1269 (83%)]	Loss: 4.013216
Train Epoche: 14 [1054/1269 (83%)]	Loss: 0.207280
Train Epoche: 14 [1055/1269 (83%)]	Loss: 0.823842
Train Epoche: 14 [1056/1269 (83%)]	Loss: 0.896073
Train Epoche: 14 [1057/1269 (83%)]	Loss: 0.982154
Train Epoche: 14 [1058/1269 (83%)]	Loss: 0.447247
Train Epoche: 14 [1059/1269 (83%)]	Loss: 0.025887
Train Epoche: 14 [1060/1269 (84%)]	Loss: 0.620142
Train Epoche: 14 [1061/1269 (84%)]	Loss: 0.057806
Train Epoche: 14 [1062/1269 (84%)]	Loss: 2.424821
Train Epoche: 14 [1063/1269 (84%)]	Loss: 0.672442
Train Epoche: 14 [1064/1269 (84%)]	Loss: 0.971423
Train Epoche: 14 [1065/1269 (84%)]	Loss: 0.000660
Train Epoche: 14 [1066/1269 (84%)]	Loss: 0.025540
Train Epoche: 14 [1067/1269 (84%)]	Loss: 0.050982
Train Epoche: 14 [1068/1269 (84%)]	Loss: 0.174872
Train Epoche: 14 [1069/1269 (84%)]	Loss: 0.707308
Train Epoche: 14 [1070/1269 (84%)]	Loss: 0.311963


In [9]:
def test():
    total = 0
    for data, target in test_data:
        model.eval()
        #files.listdir(path)
        data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0)
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        out = model(data).cpu()
        #print(out)
        out = out.detach().numpy()
        out = np.round(out)
        target = target.cpu()
        target = target.detach().numpy()
        #print(data)
        #print(data["driverId"])
        total += abs(out - target[0][0])
        print("current_position: ", data[0][0].item())
        print("Output: ", out)
        print("Target: ", target)
        print("Difference: ", out - target)
    print("Total:", total)
test()

Total: 0
